In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import kendalltau
import seaborn as sns
from sklearn.feature_selection import SelectKBest, mutual_info_classif, chi2, RFE
from sklearn.ensemble import RandomForestClassifier
from IPython.core.display_functions import display
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV

from sklearn.model_selection import GridSearchCV



from sklearn.metrics import r2_score

In [ ]:
df = pd.read_csv("encoded_df.csv")

In [ ]:
display(df)

In [ ]:
from sklearn.model_selection import GridSearchCV

X = df.drop('total_sales_price', axis=1)
y = df['total_sales_price']

# split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize random forest regressor
rf = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=6)

rf.fit(X_train, y_train)

# grid search
# param_grid = {
#     'n_estimators': [100, 200],
#     'max_depth': [None, 12],
#     'min_samples_split': [2, 10],
#     'min_samples_leaf': [1, 4],
#     'max_features': ['auto', 'sqrt']
# }
#
# random_search = RandomizedSearchCV(
#     estimator=rf,
#     param_distributions=param_grid,
#     n_iter=25,  # Number of parameter combinations to sample
#     cv=5,
#     scoring='neg_mean_squared_error',
#     verbose=2,
#     n_jobs=-1,
#     random_state=42,
# )
# random_search.fit(X_train, y_train)
#
# best_rf = random_search.best_estimator_
# y_pred = best_rf.predict(X_test)

# end grid search

y_pred = rf.predict(X_test)

target_mean = df['total_sales_price'].mean()
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Target sales price mean: {target_mean:.2f}")
print(f"Mean Absolute Error: {mae:.2f}")
print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R-squared: {r2:.2f}")

In [ ]:

# get first tree
first_tree = rf.estimators_[0]

plt.figure(figsize=(20, 10))

# plot first tree
plot_tree(first_tree, filled=True, feature_names=X.columns, fontsize=12)

plt.show()

In [ ]:
def bucketize_data(y_true, y_pred, num_buckets):
    # creates a dataframe with the true vales and predicted values
    df_bucket = pd.DataFrame({'true_values': y_true, 'predicted_values': y_pred})

    # assign each observation to a bucket based on quantiles
    df_bucket['bucket'], bucket_edges = pd.qcut(df_bucket['true_values'], q=num_buckets, labels=False, retbins=True)

    # group by bucket and calculate the average true and predicted values for each bucket
    bucket_comparison = df_bucket.groupby('bucket').agg(
        true_avg=('true_values', 'mean'),
        predicted_avg=('predicted_values', 'mean'),
        count=('bucket', 'count')
    )

    return bucket_comparison, bucket_edges

# number of buckets we want
num_buckets = 10

# calculate the bucket comparison and bucket edges
bucket_comparison, bucket_edges = bucketize_data(y_test, y_pred, num_buckets)

# display the bucket comparison
display(bucket_comparison)

# plot the average true values and predicted values for each bucket
plt.figure(figsize=(12, 6))
plt.plot(bucket_comparison.index, bucket_comparison['true_avg'], label='True Averages', marker='o')
plt.plot(bucket_comparison.index, bucket_comparison['predicted_avg'], label='Predicted Averages', marker='o')
plt.xlabel('Buckets')
plt.ylabel('Averages')
plt.title('Comparison of Predicted Averages and True Averages for Each Bucket')
plt.legend()
plt.grid()
plt.show()